In [24]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [36]:
df = pd.read_csv('world_data_full_apply_nulos.csv', index_col=0)
df.head(10)

,country,density,abbreviation,agricultural_land,land_area,armed_forces_size,birth_rate,calling_code,capital/major_city,co2-emissions,cpi,cpi_change,currency-code,fertility_rate,forested_area,gasoline_price,gdp,gross_primary_education_enrollment,gross_tertiary_education_enrollment,infant_mortality,largest_city,life_expectancy,maternal_mortality_ratio,minimum_wage,official_language,out_of_pocket_health_expenditure,physicians_per_thousand,population,population_labor_force_participation,tax_revenue,total_tax_rate,unemployment_rate,urban_population,latitude,longitude,continent
0,Afghanistan,60.0,AF,58.1,652.230,323.0,32.49,93.0,Kabul,8.672,149.90,2.3,AFN,4.47,2.1,0.70,1.910135e+10,104.0,9.7,47.9,Kabul,64.5,638.0,0.43,Pashto,78.4,0.28,NaN,48.9,9.3,71.4,11.12,NaN,33.939110,67.709953,Asia
1,Albania,105.0,AL,43.1,28.748,9.0,11.78,355.0,Tirana,4.536,119.05,1.4,ALL,1.62,28.1,1.36,1.527808e+10,107.0,55.0,7.8,Tirana,78.5,15.0,1.12,Albanian,56.9,1.20,NaN,55.7,18.6,36.6,12.33,NaN,41.153332,20.168331,Europe
2,Algeria,18.0,DZ,17.4,NaN,317.0,24.28,213.0,Algiers,150.006,151.36,2.0,DZD,3.02,0.8,0.28,1.699882e+11,109.9,51.4,20.1,Algiers,76.7,112.0,0.95,Arabic,28.1,1.72,NaN,41.2,37.2,66.1,11.70,NaN,28.033886,1.659626,Africa
3,Andorra,164.0,AD,40.0,468.000,NaN,7.20,376.0,Andorra la Vella,469.000,NaN,NaN,EUR,1.27,34.0,1.51,3.154058e+09,106.4,NaN,2.7,Andorra la Vella,NaN,NaN,6.63,Catalan,36.4,3.33,77.142,NaN,NaN,NaN,NaN,67.873,42.506285,1.521801,Europe
4,Angola,26.0,AO,47.5,NaN,117.0,40.73,244.0,Luanda,34.693,261.73,17.1,AOA,5.52,46.3,0.97,9.463542e+10,113.5,9.3,51.6,Luanda,60.8,241.0,0.71,Portuguese,33.4,0.21,NaN,77.5,9.2,49.1,6.89,NaN,-11.202692,17.873887,Africa
5,Antigua and Barbuda,223.0,AG,20.5,443.000,0.0,15.33,1.0,"St. John's, Saint John",557.000,113.81,1.2,XCD,1.99,22.3,0.99,1.727759e+09,105.0,24.8,5.0,"St. John's, Saint John",76.9,42.0,3.04,English,24.3,2.76,97.118,NaN,16.5,43.0,NaN,23.800,17.060816,-61.796428,Central America
6,Argentina,17.0,AR,54.3,NaN,105.0,17.02,54.0,Buenos Aires,201.348,232.75,53.5,ARS,2.26,9.8,1.10,4.496634e+11,109.7,90.0,8.8,Buenos Aires,76.5,39.0,3.35,Spanish,17.6,3.96,NaN,61.3,10.1,106.3,9.79,NaN,-38.416097,-63.616672,South America
7,Armenia,104.0,AM,58.9,29.743,49.0,13.99,374.0,Yerevan,5.156,129.18,1.4,AMD,1.76,11.7,0.77,1.367280e+10,92.7,54.6,11.0,Yerevan,74.9,26.0,0.66,Armenian,81.6,4.40,NaN,55.6,20.9,22.6,16.99,NaN,40.069099,45.038189,Asia
8,Australia,3.0,AU,48.2,NaN,58.0,12.60,61.0,Canberra,375.908,119.80,1.6,AUD,1.74,16.3,0.93,1.392681e+12,100.3,113.1,3.1,Sydney,82.7,6.0,13.59,NaN,19.6,3.68,NaN,65.5,23.0,47.4,5.27,NaN,-25.274398,133.775136,Oceania
9,Austria,109.0,AT,32.4,83.871,21.0,9.70,43.0,Vienna,61.448,118.06,1.5,EUR,1.47,46.9,1.20,4.463147e+11,103.1,85.1,2.9,Vienna,81.6,5.0,NaN,German,17.9,5.17,NaN,60.7,25.4,51.4,4.67,NaN,47.516231,14.550072,Europe



1. Evalúa si las siguientes afirmaciones corresponden a una distribución exponencial o poisson.
  - ¿El tiempo que transcurre entre la llegada de autobuses a una parada sigue una distribución exponencial o de Poisson?
        Se trata de una distribución exponencial --> tiempo continuo entre eventos

  - ¿El tiempo que transcurre entre la llegada de solicitudes de servicio en un restaurante sigue una distribución exponencial o de Poisson?
        El mismo que el anterior

  - ¿El tiempo que transcurre entre la llegada de taxis en una parada sigue una distribución exponencial o de Poisson?
        El mismo que la anterior

  - ¿El número de errores de ortografía en un libro de texto sigue una distribución exponencial o de Poisson?
      Distribución de Poisson --> cuentan la cantidad de eventos (faltas ortográficas) dentro de un intervalo fijo (libro)

2. Ejercicios intervalo de confianza. Al igual que en el caso del ejercicio de la clase invertida, piensa un motivo por el que sería interesante calcular estos intervalos de confianza para la extracción de insights.
  - Intervalo de Confianza para la Esperanza de Vida
    Contexto: Trabajas en una organización de salud global y deseas estimar el intervalo de confianza al 95% para la esperanza de vida promedio de cada continente.
    Nota: Cuidado porque puede que os salgan algunos nulos, genera un código para que no salgan nulos en los intervalos de confianza.

  - Intervalo de Confianza para el PIB
    Contexto: Eres una economista y quieres calcular el intervalo de confianza al 90% para el Producto Interno Bruto (PIB) de los continentes.

In [30]:
import scipy.stats as stats

In [31]:
# 2.1. 

# lo primero que tenemos que hacer es calcular la estimación puntual, en este caso calcularemos la media de la duración de las llamadas
media_duracion = df.groupby('continent')['life_expectancy'].mean().reset_index()


# después vamos a calcular el error estándar utilizando el método "sem" de la librería stats
error_duracion = stats.sem(df["life_expectancy"])

# definimos el nivel de confianza (95% en este caso)
nivel_confianza_duracion = 0.95

# calculamos los grados de libertad de la muestra. Recordad que debemos restar el total de datos que tenemos -1. 
grados_libertad_duracion = len(df["life_expectancy"]) - 1

# calculamos el valor crítico de la muestra
valor_critico_duracion = stats.t.ppf((1 + nivel_confianza_duracion) / 2, df=grados_libertad_duracion)

# calculamos el intervalo de confianza
limite_inferior_duracion = media_duracion - valor_critico_duracion * error_duracion
limite_superior_duracion = media_duracion + valor_critico_duracion * error_duracion

TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [ ]:
print("Intervalo de Confianza para la Esperanza de Vida:")
print(f"Media Muestral: {np.round(media_duracion, 2)}")
print(f"Error Estándar: {np.round(error_duracion, 2)}")
print(f"Nivel de Confianza: {nivel_confianza_duracion}")
print(f"Valor Crítico: {np.round(valor_critico_duracion, 2)}")
print(f"Intervalo de Confianza: ({np.round(limite_inferior_duracion, 2)}, {np.round(limite_superior_duracion, 2)})")

Intervalo de Confianza para la Duración de Llamadas Telefónicas:
Media Muestral: 72.28
Error Estándar: nan
Nivel de Confianza: 0.95
Valor Crítico: 1.97
Intervalo de Confianza: (nan, nan)


In [12]:
def calcular_ic_por_continente(df, columna, continent, nivel_confianza=0.95):
    resultados = []
    
    for continente, datos in df.groupby(continent):
        muestra = datos[columna].dropna()  # Eliminar nulos de cada grupo
        n = len(muestra)
        if n > 1:  # Solo calcular si hay más de un dato
            media = muestra.mean()
            error_estandar = stats.sem(muestra)
            grados_libertad = n - 1
            valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)
            
            limite_inferior = media - valor_critico * error_estandar
            limite_superior = media + valor_critico * error_estandar
            
            resultados.append({
                continent: continente,
                "Media": np.round(media, 2),
                "Error Estándar": np.round(error_estandar, 2),
                "Valor Crítico": np.round(valor_critico, 2),
                "IC Inferior": np.round(limite_inferior, 2),
                "IC Superior": np.round(limite_superior, 2)
            })
    
    return pd.DataFrame(resultados)

# Aplicar la función para obtener el intervalo de confianza para cada continente
ic_por_continente = calcular_ic_por_continente(df, 'life_expectancy', 'continent')

# Mostrar los resultados
print(ic_por_continente)

         continent  Media  Error Estándar  Valor Crítico  IC Inferior  \
0           Africa  63.72            0.83           2.01        62.05   
1             Asia  73.78            0.72           2.02        72.32   
2  Central America  74.69            0.79           2.09        73.04   
3           Europe  79.20            0.55           2.02        78.09   
4    North America  80.20            1.70          12.71        58.60   
5          Oceania  71.12            1.69           2.20        67.40   
6    South America  74.82            0.84           2.18        72.99   

   IC Superior  
0        65.40  
1        75.24  
2        76.34  
3        80.32  
4       101.80  
5        74.85  
6        76.64  


In [13]:
# Agrupamos por continente y calculamos estadísticas básicas
grouped = df.groupby('continent')['life_expectancy']
stats = grouped.agg(['mean', 'std', 'count']).reset_index()
# Agregamos el margen de error e intervalo de confianza
z = 0.975  # Valor crítico para un nivel de confianza del 95%
stats['margin_of_error'] = z * (stats['std'] / np.sqrt(stats['count']))
stats['ci_lower'] = stats['mean'] - stats['margin_of_error']
stats['ci_upper'] = stats['mean'] + stats['margin_of_error']
# Mostramos los resultados
print(stats[['continent', 'mean', 'ci_lower', 'ci_upper']])

         continent       mean   ci_lower   ci_upper
0           Africa  63.725000  62.913637  64.536363
1             Asia  73.781818  73.077085  74.486551
2  Central America  74.690000  73.921734  75.458266
3           Europe  79.202326  78.663894  79.740757
4    North America  80.200000  78.542500  81.857500
5          Oceania  71.125000  69.473763  72.776237
6    South America  74.815385  73.997191  75.633578


In [33]:
import scipy.stats as stats

In [38]:
# 2.2.

def calcular_ic_por_continente(df, columna, continent, nivel_confianza=0.90):
    resultados = []
    
    for continente, datos in df.groupby(continent):
        muestra = datos[columna].dropna()  # Eliminar nulos de cada grupo
        n = len(muestra)
        if n > 1:  # Solo calcular si hay más de un dato
            media = muestra.mean()
            error_estandar = stats.sem(muestra)
            grados_libertad = n - 1
            valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)
            
            limite_inferior = media - valor_critico * error_estandar
            limite_superior = media + valor_critico * error_estandar
            
            resultados.append({
                continent: continente,
                "Media": np.round(media, 2),
                "Error Estándar": np.round(error_estandar, 2),
                "Valor Crítico": np.round(valor_critico, 2),
                "IC Inferior": np.round(limite_inferior, 2),
                "IC Superior": np.round(limite_superior, 2)
            })
    
    return pd.DataFrame(resultados)

# Aplicar la función para obtener el intervalo de confianza para cada continente
ic_por_continente = calcular_ic_por_continente(df, 'gdp', 'continent')

# Mostrar los resultados
print(ic_por_continente)

         continent         Media  Error Estándar  Valor Crítico   IC Inferior  \
0           Africa  4.608056e+10    1.205256e+10           1.67  2.589627e+10   
1             Asia  8.371953e+11    4.631673e+11           1.68  5.857826e+10   
2  Central America  8.847993e+10    6.200288e+10           1.73 -1.873129e+10   
3           Europe  4.867482e+11    1.241827e+11           1.68  2.781923e+11   
4    North America  1.158206e+13    9.845637e+12           6.31 -5.058084e+13   
5          Oceania  1.167886e+11    9.923009e+10           1.77 -5.894132e+10   
6    South America  2.984337e+11    1.367245e+11           1.78  5.475128e+10   

    IC Superior  
0  6.626485e+10  
1  1.615812e+12  
2  1.956911e+11  
3  6.953041e+11  
4  7.374497e+13  
5  2.925184e+11  
6  5.421161e+11  


# BONUS # 

- Intervalo de Confianza para la Tasa de Natalidad
  Contexto: Trabajas en una agencia gubernamental que se ocupa de cuestiones demográficas y deseas determinar el intervalo de confianza al 99% para la tasa de natalidad promedio en Asia.

- Intervalo de Confianza para la Tasa de Desempleo
  Contexto: Eres un analista de políticas públicas y deseas calcular el intervalo de confianza al 95% para la tasa de desempleo promedio en los países de Oceanía.

- Intervalo de Confianza para la Tasa de Impuestos
  Contexto: Trabajas en un ministerio de finanzas y deseas estimar el intervalo de confianza al 90% para la tasa de impuestos promedio en los países de África.

In [28]:
import scipy.stats as stats

In [ ]:
# BONUS 1

# Filtro para Asia
df_asia = df[df['continent'] == 'Asia']

# Eliminar filas con valores nulos en la tasa de natalidad
df_asia_clean = df_asia.dropna(subset=['birth_rate'])

# Cálculo de la media y el error estándar
mean_birth_rate = df_asia_clean['birth_rate'].mean()

# Aquí es donde ocurre el error. Usamos la columna 'birth_rate' de forma adecuada
sem_birth_rate = stats.sem(df_asia_clean['birth_rate'])  # Usar df_asia_clean en lugar de df_asia

# Definir el nivel de confianza (99% en este caso)
confidence_level = 0.99

# Calcular los grados de libertad
df_degrees_freedom = len(df_asia_clean['birth_rate']) - 1

# Calcular el valor crítico de la t de Student
t_critical = stats.t.ppf((1 + confidence_level) / 2, df=df_degrees_freedom)

# Calcular el intervalo de confianza
ci_lower = mean_birth_rate - t_critical * sem_birth_rate
ci_upper = mean_birth_rate + t_critical * sem_birth_rate

# Resultado
print(f"Intervalo de Confianza para la Tasa de Natalidad en Asia:")
print(f"Media: {np.round(mean_birth_rate, 2)}")
print(f"Error Estándar: {np.round(sem_birth_rate, 4)}")
print(f"Nivel de Confianza: {confidence_level}")
print(f"Valor Crítico: {np.round(t_critical, 2)}")
print(f"Intervalo de Confianza: ({np.round(ci_lower, 2)}, {np.round(ci_upper, 2)})")

Intervalo de Confianza para la Tasa de Natalidad en Asia:
Media: 18.81
Error Estándar: 0.991
Nivel de Confianza: 0.99
Valor Crítico: 2.7
Intervalo de Confianza: (16.14, 21.48)


In [ ]:
import scipy.stats as stats

In [40]:
# BONUS 2
# Filtro para el continente de interés (por ejemplo, Asia)
df_continente = df[df['continent'] == 'Oceania']

# Eliminar filas con valores nulos en la columna PIB
df_continente_clean = df_continente.dropna(subset=['unemployment_rate'])

# Cálculo de la media y el error estándar
mean_unemployment_rate = df_continente_clean['unemployment_rate'].mean()
sem_unemployment_rate = stats.sem(df_continente_clean['unemployment_rate'])

# Definir el nivel de confianza (90% en este caso)
confidence_level = 0.90

# Calcular los grados de libertad
df_degrees_freedom = len(df_continente_clean['unemployment_rate']) - 1

# Calcular el valor crítico de la t de Student
t_critical = stats.t.ppf((1 + confidence_level) / 2, df=df_degrees_freedom)

# Calcular el intervalo de confianza
ci_lower = mean_unemployment_rate - t_critical * sem_unemployment_rate
ci_upper = mean_unemployment_rate + t_critical * sem_unemployment_rate

# Resultado
print(f"Intervalo de Confianza para la Tasa de Desempleo en Oceanía:")
print(f"Media del PIB: {np.round(mean_unemployment_rate, 2)}")
print(f"Error Estándar: {np.round(sem_unemployment_rate, 2)}")
print(f"Nivel de Confianza: {confidence_level}")
print(f"Valor Crítico: {np.round(t_critical, 2)}")
print(f"Intervalo de Confianza: ({np.round(ci_lower, 2)}, {np.round(ci_upper, 2)})")

Intervalo de Confianza para la Tasa de Desempleo en Oceanía:
Media del PIB: 3.79
Error Estándar: 0.87
Nivel de Confianza: 0.9
Valor Crítico: 1.89
Intervalo de Confianza: (2.14, 5.45)


In [41]:
import scipy.stats as stats

In [47]:
# BONUS 3

# BONUS 2
# Filtro para el continente de interés (por ejemplo, Asia)
df_continente = df[df['continent'] == 'Africa']

# Eliminar filas con valores nulos en la columna PIB
df_continente_clean = df_continente.dropna(subset=['unemployment_rate'])

# Cálculo de la media y el error estándar
mean_unemployment_rate = df_continente_clean['unemployment_rate'].mean()
sem_unemployment_rate = stats.sem(df_continente_clean['unemployment_rate'])

# Definir el nivel de confianza (90% en este caso)
confidence_level = 0.90

# Calcular los grados de libertad
df_degrees_freedom = len(df_continente_clean['unemployment_rate']) - 1

# Calcular el valor crítico de la t de Student
t_critical = stats.t.ppf((1 + confidence_level) / 2, df=df_degrees_freedom)

# Calcular el intervalo de confianza
ci_lower = mean_unemployment_rate - t_critical * sem_unemployment_rate
ci_upper = mean_unemployment_rate + t_critical * sem_unemployment_rate

# Resultado
print(f"Intervalo de Confianza para la Tasa de Desempleo en Oceanía:")
print(f"Media del PIB: {np.round(mean_unemployment_rate, 2)}")
print(f"Error Estándar: {np.round(sem_unemployment_rate, 2)}")
print(f"Nivel de Confianza: {confidence_level}")
print(f"Valor Crítico: {np.round(t_critical, 2)}")
print(f"Intervalo de Confianza: ({np.round(ci_lower, 2)}, {np.round(ci_upper, 2)})")

Intervalo de Confianza para la Tasa de Desempleo en Oceanía:
Media del PIB: 7.88
Error Estándar: 0.9
Nivel de Confianza: 0.9
Valor Crítico: 1.68
Intervalo de Confianza: (6.38, 9.39)


In [43]:
import scipy.stats as stats

In [ ]:
# BONUS 3


# Filtrar los países de África
df_africa = df[df['continent'] == 'Africa']

# Asegurarse de que la tasa de impuestos sea numérica
df_africa['total_tax_rate'] = pd.to_numeric(df_africa['total_tax_rate'], errors='coerce')

# Eliminar filas con valores nulos en la tasa de impuestos
df_africa_clean = df_africa.dropna(subset=['total_tax_rate'])

# Cálculo de la media y el error estándar
mean_tax_rate = df_africa_clean['total_tax_rate'].mean()
sem_tax_rate = stats.sem(df_africa_clean['total_tax_rate'])

# Definir el nivel de confianza (90% en este caso)
confidence_level = 0.90

# Calcular los grados de libertad
df_degrees_freedom = len(df_africa_clean['total_tax_rate']) - 1

# Calcular el valor crítico de la t de Student
t_critical = stats.t.ppf((1 + confidence_level) / 2, df=df_degrees_freedom)

# Calcular el intervalo de confianza
ci_lower = mean_tax_rate - t_critical * sem_tax_rate
ci_upper = mean_tax_rate + t_critical * sem_tax_rate

# Resultado
print(f"Intervalo de Confianza para la Tasa de Impuestos en África:")
print(f"Media de la Tasa de Impuestos: {np.round(mean_tax_rate, 2)}")
print(f"Error Estándar: {np.round(sem_tax_rate, 2)}")
print(f"Nivel de Confianza: {confidence_level}")
print(f"Valor Crítico: {np.round(t_critical, 2)}")
print(f"Intervalo de Confianza: ({np.round(ci_lower, 2)}, {np.round(ci_upper, 2)})")


Intervalo de Confianza para la Tasa de Impuestos en África:
Media de la Tasa de Impuestos: 47.77
Error Estándar: 4.04
Nivel de Confianza: 0.9
Valor Crítico: 1.68
Intervalo de Confianza: (41.0, 54.54)
